In [1]:
from src.tsp_utils import TSPUtils
from src.tsp_animator import TSPAnimator
from src.algorithms.ant_colony import AntColony

def run():
    """
    Run the Genetic Algorithm on the Traveling Salesman Problem (TSP)
    and visualize the optimization process.
    """
    # Parameters
    cities = 20
    population = 20
    generations = 200

    # Initialize problem
    generated = TSPUtils.generate_cities(cities)
    algorithm = AntColony(generated, population, generations)
    history = algorithm.run()

    # Visualize results
    animator = TSPAnimator(generated, history, algorithm_name="AntColony", output_dir="../assets/AntColony/")
    animator.save()


run()

Generation 1/200, Best Path Distance: 516.12
Generation 2/200, Best Path Distance: 455.39
Generation 3/200, Best Path Distance: 490.29
Generation 4/200, Best Path Distance: 468.13
Generation 5/200, Best Path Distance: 419.56
Generation 6/200, Best Path Distance: 460.81
Generation 7/200, Best Path Distance: 403.10
Generation 8/200, Best Path Distance: 414.02
Generation 9/200, Best Path Distance: 426.06
Generation 10/200, Best Path Distance: 371.14
Generation 11/200, Best Path Distance: 379.66
Generation 12/200, Best Path Distance: 387.62
Generation 13/200, Best Path Distance: 351.20
Generation 14/200, Best Path Distance: 351.69
Generation 15/200, Best Path Distance: 379.08
Generation 16/200, Best Path Distance: 376.39
Generation 17/200, Best Path Distance: 351.20
Generation 18/200, Best Path Distance: 381.40
Generation 19/200, Best Path Distance: 373.80
Generation 20/200, Best Path Distance: 381.40
Generation 21/200, Best Path Distance: 351.20
Generation 22/200, Best Path Distance: 351.